# [Keras + Universal Sentence Encoder = Deep Meter](https://www.dlology.com/blog/keras-meets-universal-sentence-encoder-transfer-learning-for-text-data/) 

This notebook creates an autoencoder using the Universal Sentence Encoder. The autoencoder output is CMUdict syllables. The dataset is that subset of Allison Parrish's Project Gutenberg poetry archive which happens to scan in iambic pentameter.

The notebook is based on Chengwei Zhang's example of wrapping the USE inside a larger tensorflow model saves to a Keras model (without save the USE itself in the TF model).

The Universal Sentence Encoder makes getting sentence level embeddings as easy as it has historically been to lookup the embeddings for individual words. The sentence embeddings can then be trivially used to compute sentence level meaning similarity as well as to enable better performance on downstream classification tasks using less supervised training data.

Since there are 10 one-hot values for 10 sets of 6k syllables, this is "multi-label classification"
Changes for multi-label classification:
sigmoid activation instead of softmax
binary_crossentropy

Text format is tab-separated, 2 columns: first text, second multi-level
array of syllables:


# Getting Started

This section sets up the environment for access to the Universal Sentence Encoder on TF Hub and provides examples of applying the encoder to words, sentences, and paragraphs.

In [1]:
# Install the latest Tensorflow version.
!pip3 install --quiet "tensorflow>=1.7"
# Install TF-Hub.
!pip3 install --quiet tensorflow-hub
%cd /content
!git clone https://github.com/LanceNorskog/deep_meter || true
%cd deep_meter
!git pull
# could not figure out how to read gzipped files as text!
!gunzip -qf blobs/*.gz || true
!gunzip -qf prepped_data/*.gz || true

/content
fatal: destination path 'deep_meter' already exists and is not an empty directory.
/content/deep_meter
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 3 (delta 2), reused 3 (delta 2), pack-reused 0
Unpacking objects: 100% (3/3), done.
From https://github.com/LanceNorskog/deep_meter
   5ea83be..7b7f079  master     -> origin/master
Updating 5ea83be..7b7f079
Fast-forward
 ...Udict_+_Project_Gutenberg_poetry_archives.ipynb | 618 +++++++++------------
 1 file changed, 270 insertions(+), 348 deletions(-)
gzip: blobs/*.gz: No such file or directory
gzip: prepped_data/*.gz: No such file or directory


In [0]:
# boilerplate from base notebook
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
import keras.layers as layers
from keras.models import Model
from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Nadam, Adam
np.random.seed(10)

In [0]:
# github deep_meter code
import utils
# should not need this to use utils.flatten but is true anyway?
from itertools import chain
import subprocess
import syllables
# misc for this notebook
from ast import literal_eval

import scipy



In [0]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]

In [7]:
# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(module_url)
embed_size = embed.get_output_info_dict()['default'].get_shape()[1].value

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.


In [0]:
# read classified poetry lines: text tab [['syll', 'la', 'ble'], ...]
# clip to only most common syllables with syllable manager
def get_data(filename, syll_mgr, num_symbols):
    lines = open(filename, 'r').read().splitlines()
    text_lines = []
    enc_array = []
    num_syllables = syll_mgr.get_size()
    for i in range(0, len(lines)):
      parts = lines[i].split("\t")
      label = utils.flatten(literal_eval(parts[1]))
      if len(label) != num_symbols:
        continue
      enc = np.zeros((num_symbols * syll_mgr.get_size()), dtype=np.int8)
      for j in range(num_symbols):
        enc[num_syllables * j + syll_mgr.get_encoding(label[j])] = 1
      text_lines.append([parts[0]])
      enc_array.append(enc)

    return (np.array(text_lines), np.array(enc_array))


In [0]:
# syllables in descending order of occurrence - 6k in gutenberg.iambic_pentameter, 15k total
# clamp to most common 100 syllables while debugging- use NCE to get all syllables or interesting number
# 98 + pause + wildcard
# iambic pentameter
num_symbols = 10
#syll_mgr = syllables.syllables(num_syllables)
syll_mgr = syllables.syllables()
num_syllables = syll_mgr.get_size() 



In [26]:
(train_text, train_label) = get_data('prepped_data/gutenberg.iambic_pentameter.train', syll_mgr, num_symbols)
num_training = len(train_text)
#train_text = train_text[0:10000]
#train_label = train_label[0:10000]

(test_text, test_label) = get_data('prepped_data/gutenberg.iambic_pentameter.test', syll_mgr, num_symbols)
num_testing = len(test_text)
train_label.shape


(58498, 66350)

In [0]:

# slow
num_epochs = 50
adam_0001 = Adam(0.0001)
adam_0001 = tf.contrib.opt.NadamOptimizer(0.0001)

## Wrap embed module in a Lambda layer
Explicitly cast the input as a string

In [0]:
def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]


In [0]:
def weighted_binary_crossentropy(positive_weights, y_pred):
    y_true = K.cast(K.greater(positive_weights, 1), K.floatx())
    loss = -positive_weights*y_true*K.log(y_pred + K.epsilon()) - (1-y_true)*K.log(1 - y_pred + K.epsilon())
    return K.mean(loss, axis=-1)

## Assemble model

In [29]:
# changed accuracy from 'choose your own accuracy'
input_text = layers.Input(shape=(1,), dtype=tf.string)
embedding = layers.Lambda(UniversalEmbedding, output_shape=(embed_size,), name='TF-Hub')(input_text)
dropout_input = layers.Dropout(0.2)(embedding)
dense = layers.Dense(768, activation='relu', name='Convoluted')(dropout_input)
dense = layers.Dropout(0.2)(dense)
dense = layers.Dense(1024, activation='relu', name='Medium')(dense)
dense = layers.Dropout(0.2)(dense)
dense = layers.Dense(2048, activation='relu', name='Smooth')(dense)
pred = layers.Dense(num_symbols * num_syllables, activation='sigmoid', name='Flatout')(dense)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='binary_crossentropy', 
              optimizer=adam_0001, 
              metrics=['binary_accuracy','categorical_accuracy'],
              loss_weights=inverse_zipf)
model.summary()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
TF-Hub (Lambda)              (None, 512)               0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
Convoluted (Dense)           (None, 768)               393984    
_________________________________________________________________
dropout_8 (Dropout)          (None, 768)               0         
_________________________________________________________________
Medium (Dense)               (None, 1024)              787456    
_____________________________________________________

## Train Keras model and save weights
This only trains and save our Keras layers not the embed module' weights.

In [30]:
use_saved_model=False
retrain=False
if not use_saved_model or not os.path.exists('./model.h5'):
  with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    if retrain:
      model.load_weights('./model.h5')  
    else:
      session.run(tf.tables_initializer())
    history = model.fit(train_text, 
            train_label,
            validation_data=(test_text, test_label),
            epochs=num_epochs,
            callbacks = [EarlyStopping(patience=2)],
            batch_size=32,
            verbose=2)
    model.save_weights('./model.h5')


Train on 58498 samples, validate on 4200 samples
Epoch 1/50
 - 285s - loss: 0.0115 - binary_accuracy: 0.9989 - categorical_accuracy: 0.2058 - val_loss: 0.0011 - val_binary_accuracy: 0.9998 - val_categorical_accuracy: 0.2283
Epoch 2/50
 - 279s - loss: 0.0011 - binary_accuracy: 0.9998 - categorical_accuracy: 0.2512 - val_loss: 0.0011 - val_binary_accuracy: 0.9998 - val_categorical_accuracy: 0.3350
Epoch 3/50
 - 279s - loss: 0.0010 - binary_accuracy: 0.9999 - categorical_accuracy: 0.3257 - val_loss: 0.0010 - val_binary_accuracy: 0.9999 - val_categorical_accuracy: 0.3414
Epoch 4/50
 - 279s - loss: 0.0010 - binary_accuracy: 0.9999 - categorical_accuracy: 0.3440 - val_loss: 0.0010 - val_binary_accuracy: 0.9999 - val_categorical_accuracy: 0.3490
Epoch 5/50
 - 278s - loss: 0.0010 - binary_accuracy: 0.9999 - categorical_accuracy: 0.3799 - val_loss: 0.0010 - val_binary_accuracy: 0.9999 - val_categorical_accuracy: 0.4360
Epoch 6/50
 - 279s - loss: 9.9142e-04 - binary_accuracy: 0.9999 - categorica

In [31]:
!ls -alh | grep model.h5

-rw-r--r-- 1 root root 532M Oct 27 11:16 model.h5


## Make predictions

In [0]:

#new_text = np.array(new_text, dtype=object)[:, np.newaxis]
with tf.Session() as session:
  K.set_session(session)
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  model.load_weights('./model.h5')  
  predicts = model.predict(test_text, batch_size=32)

In [33]:
first = predicts[0]
max_d = -1000000
min_d = 1000000
max_i = 0
min_i = 0
for i in range(len(predicts) - 1):
  v = predicts[i]
  d = scipy.spatial.distance.cosine(v, first)
  if max_d < d:
    max_d = d
    max_i = i
  if min_d > d:
    min_d = d
    min_i = i
print("Min and max distances: {0} and {1}".format(min_d, max_d))
print(test_text[max_i])
syll_mgr.interpret2(predicts[max_i])

Min and max distances: 2.6573251787898045e-07 and 0.9630461096219043
['And found his whole good-breeding scarce enough.']
[7, 480, 10, 958, 36, 62, 10, 10, 36, 2149]
['AE N D', 'S K EH R S', 'HH IH Z', 'HH IY', 'AH', 'F AO R', 'HH IH Z', 'HH IH Z', 'AH', 'N IY D']


In [34]:
for i in range(10,100):
  print(test_text[i])
  syll_mgr.interpret2(predicts[i])
  print('')

['Of humble seeming: and, behind them all,']
[40, 1514, 490, 7, 7, 1514, 7, 1514, 36, 3381]
['AH V', 'HH AH M', 'B AH L', 'AE N D', 'AE N D', 'HH AH M', 'AE N D', 'HH AH M', 'AH', 'M AY R']

['Behind his head upon the shoulders lay,']
[36, 996, 10, 1436, 151, 445, 2079, 1961, 10, 483]
['AH', 'B AH V', 'HH IH Z', 'N EH K', 'HH IH M', 'HH EH D', 'R HH EH D', 'SH OW L', 'HH IH Z', 'L EY']

['Discover countries, with a kindred heart']
[36, 1463, 31, 417, 117, 7, 117, 417, 117, 417]
['AH', 'S K AH', 'DH AH', 'W ER L D', 'SH AH N Z', 'AE N D', 'SH AH N Z', 'W ER L D', 'SH AH N Z', 'W ER L D']

['Was open in his praise, and plaudits rose']
[156, 1398, 7, 453, 7, 7, 10, 776, 156, 671]
['W AA Z', 'JH OY D', 'AE N D', 'F EY TH', 'AE N D', 'AE N D', 'HH IH Z', 'G R EY', 'W AA Z', 'HH AA R T']

['A spear the hero bore of wondrous strength,']
[36, 166, 403, 685, 31, 91, 36, 685, 40, 91]
['AH', 'M AY', 'T IY', 'S AO R D', 'DH AH', 'S P IH R', 'AH', 'S AO R D', 'AH V', 'S P IH R']

['Were things indi

In [35]:
categories = df_train.label.cat.categories.tolist()
predict_logits = predicts.argmax(axis=1)
print("Categorie: {0}".format(categories))
predict_labels = [categories[logit] for logit in predict_logits]
predict_labels

NameError: ignored

In [0]:

os.remove('./model.h5')